In [1]:
import pandas as pd
import sqlalchemy as db

!pip install psycopg2-binary
import psycopg2

from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

from sklearn.preprocessing import OneHotEncoder

In [2]:
conn = psycopg2.connect(database="postgres",
						user='postgres', password='demo1234',
						host='data-miners.c1j77v6xb2do.us-east-1.rds.amazonaws.com', port='5432'
)

conn.autocommit = True
cursor = conn.cursor()

In [3]:
ev_sql = '''SELECT * FROM public.ev_stations;'''
reg_sql = '''SELECT * FROM public.ev_registration;'''
census_sql = '''SELECT * FROM public.census_2019_income;'''

stations_df = pd.read_sql(ev_sql, con=conn)
reg_data = pd.read_sql(reg_sql, con=conn)
median_df = pd.read_sql(census_sql, con=conn)

# Preprocessing ev_stations data

In [4]:
stations_df

,station_name,street_address,city,state_code,zip_code,status_code,access_days_time,ev_level1_evse_num,ev_level2_evse_num,ev_dc_fast_count,ev_network,latitude,longitude,ev_id,owner_type_code,open_date,access_code,facility_type,charging_cost
0,LADWP - Truesdale Center,11797 Truesdale St,Sun Valley,CA,91352,E,Fleet use only,NaN,39.0,3.0,Non-Networked,34.24831915,-118.3879714,1517,LG,1999-10-15,private,UTILITY,None
1,LADWP - West LA District Office,1394 S Sepulveda Blvd,Los Angeles,CA,90024,E,None,NaN,4.0,NaN,Non-Networked,34.052542,-118.448504,1519,LG,2020-02-28,private,UTILITY,Free
2,Los Angeles Convention Center,1201 S Figueroa St,Los Angeles,CA,90015,E,24 hours daily; pay lot,NaN,12.0,NaN,Non-Networked,34.040539,-118.271387,1523,P,1995-08-30,public,PARKING_GARAGE,None
3,LADWP - John Ferraro Building,111 N Hope St,Los Angeles,CA,90012,E,For fleet and employee use only,NaN,311.0,2.0,Non-Networked,34.059133,-118.248589,1525,LG,1999-10-15,private,UTILITY,None
4,LADWP - Haynes Power Plant,6801 E 2nd St,Long Beach,CA,90803,E,Fleet use only,NaN,19.0,1.0,Non-Networked,33.759802,-118.096665,1531,LG,2018-05-01,private,UTILITY,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50216,VIP LOT STATION1,1501 Kirkwood Meadows Dr,Markleeville,CA,96120,E,24 hours daily,NaN,2.0,NaN,ChargePoint Network,38.68466,-120.065169,204783,None,2022-01-14,public,None,None
50217,Prunedale Shopping Center,8065 San Miguel Canyon Rd,Salinas,CA,93907,E,24 hours daily,NaN,NaN,6.0,eVgo Network,36.801716,-121.664153,204784,None,2022-01-14,public,None,None
50218,Beaverton Electric Avenue,11665 SW Beaverton Hillsdale Hwy,Beaverton,OR,97005,E,24 hours daily,NaN,NaN,2.0,Greenlots,45.48903,-122.798151,204785,None,2022-01-14,public,None,None
50219,Shell - Inman,2040 Highway 292,Inman,SC,29349,E,24 hours daily,NaN,NaN,1.0,Greenlots,35.082476,-82.058433,204786,None,2022-01-14,public,None,None


In [5]:
# Drop columns with unusable date
stations_df = stations_df.drop(columns=['status_code', 'access_days_time', 'ev_id', 'owner_type_code', 'open_date', 'access_code', 'facility_type', 'charging_cost'])

# bin network type
other_network = ['Volta', 'EV Connect', 'OpConnect', 'SemaCharge Network', 'Tesla Destination',
                 'Greenlots', 'EVGATEWAY', 'POWERFLEX', 'eVgo Network', 'Webasto', 'CHARGELAB',
                 'AMPUP', 'EVCS', 'Blink Network', 'FCN', 'Tesla', 'Electrify America', 'FLO',
                 'FPLEV', 'ZEFNET', 'LIVINGSTON', 'Electrify Canada']

for network in other_network:
  stations_df.ev_network = stations_df.ev_network.replace(network, 'Other-Network')

stations_df.ev_network = stations_df.ev_network.replace('ChargePoint Network', 'ChargePoint')

# drop rows with States not used in current model
states = ['MI', 'MN', 'NJ', 'NY', 'OR', 'TX', 'WA', 'WI']

stations_df = stations_df[stations_df.state_code.isin(states) == True]

# FUTURE OPTION - Classify Charging_Cost as FREE or PAID - Need solution to classify NaNs
# stations_df['Charging_Cost'] = stations_df['Charging_Cost'].replace(to_replace = ('Free', 'Paid'), value = ('FREE', 'PAID'))

# replace NaNs in EVSE type columns
stations_df['ev_level1_evse_num'] = stations_df['ev_level1_evse_num'].fillna(0)
stations_df['ev_level2_evse_num'] = stations_df['ev_level2_evse_num'].fillna(0)
stations_df['ev_dc_fast_count'] = stations_df['ev_dc_fast_count'].fillna(0)

# reorder columns in table
stations_df = stations_df[['station_name', 'street_address', 'city', 'state_code', 'zip_code', 'latitude', 'longitude', 'ev_level1_evse_num', 'ev_level2_evse_num', 'ev_dc_fast_count', 'ev_network']]

# drop rows with nan values
stations_df = stations_df.dropna().reset_index(drop=True)

# rename column names
stations_df = stations_df.rename(columns={
    'zip_code': 'ZIP_CODE',
    'ev_level1_evse_num': 'EV_LEVEL_1',
    'ev_level2_evse_num': 'EV_LEVEL_2',
    'ev_dc_fast_count': 'EV_FAST',
    'ev_network': 'NETWORK_TYPE'})

# OneHotEncode Network Type
enc = OneHotEncoder(sparse=False)
encode_df = pd.DataFrame(enc.fit_transform(stations_df.NETWORK_TYPE.values.reshape(-1,1)))
encode_df.columns = enc.get_feature_names_out(['NETWORK_TYPE'])

stations_df = stations_df.merge(encode_df, left_index=True, right_index=True).drop('NETWORK_TYPE', 1)

# rename column names
stations_df = stations_df.rename(columns={
    'NETWORK_TYPE_Non-Networked': 'NETWORK_TYPE_Non_Networked',
    'NETWORK_TYPE_Other-Network': 'NETWORK_TYPE_Other_Network'})

stations_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

,station_name,street_address,city,state_code,ZIP_CODE,latitude,longitude,EV_LEVEL_1,EV_LEVEL_2,EV_FAST,NETWORK_TYPE_ChargePoint,NETWORK_TYPE_Non_Networked,NETWORK_TYPE_Other_Network
0,City of Lacey - City Hall Parking,420 College St,Lacey,WA,98503,47.044011,-122.822404,0.0,4.0,0.0,0.0,1.0,0.0
1,Oregon Museum of Science and Industry (OMSI) -...,1945 SE Water Ave,Portland,OR,97214,45.508396,-122.664667,2.0,2.0,0.0,0.0,1.0,0.0
2,Seattle-Tacoma International Airport - General...,17801 Pacific Hwy S,Seattle,WA,98188,47.443377,-122.296229,36.0,0.0,0.0,0.0,1.0,0.0
3,Travis Park United Methodist Church,230 E Travis St,San Antonio,TX,78205,29.427254,-98.490136,2.0,0.0,0.0,0.0,1.0,0.0
4,City of Keizer - City Hall,930 Chemawa Rd NE,Salem,OR,97303,44.99574,-123.019758,2.0,0.0,0.0,0.0,1.0,0.0


# Preprocessing Registration Data

In [6]:
# function to create dataframes from csvs
def clean_state(state):

  state_df = state

  state_df = state_df.zip_code.value_counts().rename_axis('ZIP_CODE').reset_index(name='REG_COUNTS')

  state_df['ZIP_CODE'] = state_df['ZIP_CODE'].astype(int)

  return state_df

# create dataframes using clean_state function
registration_df = clean_state(reg_data)

In [7]:
registration_df

,ZIP_CODE,REG_COUNTS
0,98052,29672
1,98033,20139
2,98115,19002
3,98004,18632
4,98006,17795
...,...,...
8345,85250,1
8346,84037,1
8347,49762,1
8348,48022,1


# Preprocessing Median Income Data

In [8]:
median_df = median_df.drop(columns=['total_households_margin_error',	'median_households_margin_error',	'mean_households_estimate_income',	'mean_households_margin_error'])
median_df = median_df[median_df.median_households_estimate_income	 != '2,500-']
median_df = median_df[median_df.median_households_estimate_income	 != '250,000+']
median_df = median_df[median_df.median_households_estimate_income	 != '-']
median_df['zip_code'] = median_df['zip_code'].astype(int)
median_df['total_households'] = median_df['total_households'].astype(int)
median_df['median_households_estimate_income'] = median_df['median_households_estimate_income'].astype(int)


# rename column names
median_df = median_df.rename(columns={
    'zip_code': 'ZIP_CODE',
    'total_households': 'TOTAL_HOUSEHOLDS',
    'median_households_estimate_income': 'MEDIAN_INCOME'})

median_df.dtypes

ZIP_CODE            int64
TOTAL_HOUSEHOLDS    int64
MEDIAN_INCOME       int64
dtype: object

In [9]:
median_df

,ZIP_CODE,TOTAL_HOUSEHOLDS,MEDIAN_INCOME
0,601,5509,14361
1,602,12740,16807
2,603,19228,16049
3,606,1946,12119
4,610,8795,19898
...,...,...,...
30807,99921,814,52188
30808,99922,117,33750
30809,99925,317,62917
30810,99926,487,56188


# Final model Dataframe

In [10]:
# Drop location info for ml model
model_df = stations_df.drop(columns=['station_name',
                                     'street_address',
                                     'city',
                                     'state_code',
                                     'latitude',
                                     'longitude'])

# count each level type by zip code
model_df = model_df.groupby(['ZIP_CODE'], as_index=False).sum()

# change data types for sorting and merging
model_df['ZIP_CODE'] = model_df['ZIP_CODE'].astype(int)
model_df['EV_LEVEL_1'] = model_df['EV_LEVEL_1'].astype(int)
model_df['EV_LEVEL_2'] = model_df['EV_LEVEL_2'].astype(int)
model_df['EV_FAST'] = model_df['EV_FAST'].astype(int)
model_df['NETWORK_TYPE_ChargePoint'] = model_df['NETWORK_TYPE_ChargePoint'].astype(int)
model_df['NETWORK_TYPE_Non_Networked'] = model_df['NETWORK_TYPE_Non_Networked'].astype(int)
model_df['NETWORK_TYPE_Other_Network'] = model_df['NETWORK_TYPE_Other_Network'].astype(int)

# mege the data fram with median_df
model_df = model_df.merge(median_df, how='left', on='ZIP_CODE')

# merge the data frame with registration_df
model_df = model_df.merge(registration_df, how='left', on='ZIP_CODE')

# fill nans with zero
model_df['REG_COUNTS'] = model_df['REG_COUNTS'].fillna(0)

model_df = model_df.dropna(how='all')

# change data type for sorting and merging
model_df['REG_COUNTS'] = model_df['REG_COUNTS'].astype(int)

# soft dataframe by zip code
model_df = model_df.sort_values(by='ZIP_CODE')

# change zip code to string and reformat
model_df['ZIP_CODE'] = model_df['ZIP_CODE'].apply(lambda x: '0' + str(x) if x < 10000 else str(x)) 

# drop duplicate zip codes
model_df = model_df.drop_duplicates(subset=['ZIP_CODE'], keep='first')

# set ZIP_CODE as index
model_df = model_df.set_index('ZIP_CODE')

#drop remaining nans
model_df = model_df.dropna(axis=0, how='any')


model_df

,EV_LEVEL_1,EV_LEVEL_2,EV_FAST,NETWORK_TYPE_ChargePoint,NETWORK_TYPE_Non_Networked,NETWORK_TYPE_Other_Network,TOTAL_HOUSEHOLDS,MEDIAN_INCOME,REG_COUNTS
ZIP_CODE,,,,,,,,,
03266,0,2,0,0,1,0,851.0,54904.0,0
07001,0,1,0,0,1,0,5022.0,69849.0,173
07003,0,12,5,4,1,3,18577.0,78034.0,800
07004,0,6,3,0,1,2,2621.0,122063.0,308
07005,0,1,10,0,1,1,5489.0,111946.0,446
...,...,...,...,...,...,...,...,...,...
99352,0,13,3,1,1,4,12877.0,81410.0,3413
99354,36,48,0,0,45,1,9175.0,73369.0,2082
99361,0,1,0,0,0,1,750.0,69118.0,100


In [11]:
model_df.dtypes

EV_LEVEL_1                      int64
EV_LEVEL_2                      int64
EV_FAST                         int64
NETWORK_TYPE_ChargePoint        int64
NETWORK_TYPE_Non_Networked      int64
NETWORK_TYPE_Other_Network      int64
TOTAL_HOUSEHOLDS              float64
MEDIAN_INCOME                 float64
REG_COUNTS                      int64
dtype: object

In [12]:
model_df

,EV_LEVEL_1,EV_LEVEL_2,EV_FAST,NETWORK_TYPE_ChargePoint,NETWORK_TYPE_Non_Networked,NETWORK_TYPE_Other_Network,TOTAL_HOUSEHOLDS,MEDIAN_INCOME,REG_COUNTS
ZIP_CODE,,,,,,,,,
03266,0,2,0,0,1,0,851.0,54904.0,0
07001,0,1,0,0,1,0,5022.0,69849.0,173
07003,0,12,5,4,1,3,18577.0,78034.0,800
07004,0,6,3,0,1,2,2621.0,122063.0,308
07005,0,1,10,0,1,1,5489.0,111946.0,446
...,...,...,...,...,...,...,...,...,...
99352,0,13,3,1,1,4,12877.0,81410.0,3413
99354,36,48,0,0,45,1,9175.0,73369.0,2082
99361,0,1,0,0,0,1,750.0,69118.0,100


In [13]:
# export model_df for the ML Model predictions.
model_df.to_csv('model_df.csv')